**Comparaision VITS/Langevin**

Data : 

Return 
- X, of dimention, n_samples, n_features
- Y of dimention n_targets  * n_samples
- theta star is true_weights



A fiare varier pour notre problème : 

- 1) Imporn_features with n_informative features pour l'exploration à montrer
- 2) effective_rank : (int)  The approximate number of singular vectors required to explain most of the input data by linear combinations. 





bias =0

A faire varier : n_informative pour le coté exploration

In [50]:
from sklearn.datasets._samples_generator import check_array, check_random_state, make_low_rank_matrix
import numpy as np
from sklearn.utils import shuffle as util_shuffle

In [51]:
def regression_model(
    n_samples=100,
    n_features=100,
    n_informative=10,
    n_targets=1,
    bias=0.0,
    effective_rank=None,
    tail_strength=0.5,
    noise=1,
    shuffle=True,
    coef=True,
    random_state=None,
):
    
    n_informative = min(n_features, n_informative)
    generator = check_random_state(random_state)

    if effective_rank is None:
        # Randomly generate a well conditioned input set
        X = generator.standard_normal(size=(n_samples, n_features))
    else:
        # Randomly generate a low rank, fat tail input set
        X = make_low_rank_matrix(
            n_samples=n_samples,
            n_features=n_features,
            effective_rank=effective_rank,
            tail_strength=tail_strength,
            random_state=generator,
        )

    ground_truth = np.zeros((n_features, n_targets))
    ground_truth[:n_informative, :] = 100 * generator.uniform(
        size=(n_informative, n_targets)
    )

    y = np.dot(X, ground_truth) + bias

    # Add noise
    if noise > 0.0:
        y += generator.normal(scale=noise, size=y.shape)

    # Randomly permute samples and features
    if shuffle:
        X, y = util_shuffle(X, y, random_state=generator)

        indices = np.arange(n_features)
        generator.shuffle(indices)
        X[:, :] = X[:, indices]
        ground_truth = ground_truth[indices]

    y = np.squeeze(y)

    if coef:
        return X, y, np.squeeze(ground_truth)
    else:
        return X, y

In [54]:
X, y, true_weights = regression_model(n_samples=200, n_features=20, n_informative=20, n_targets=1, 
 bias=0.0, effective_rank=None, 
 tail_strength=0.5, noise=0.0, 
 shuffle=True, coef=True, random_state=None)

In [55]:
X.shape, y.shape

((200, 20), (200,))

In [56]:
true_weights

array([10.34039475, 72.65012421, 87.33781205,  2.28149344, 54.96317257,
       59.20795128, 16.82041862, 28.29000988, 28.29327397, 76.29703411,
       12.05209099, 18.1921814 , 87.96194328, 41.57946972, 44.02657875,
       99.53411171, 38.77696597, 32.78147825, 48.94694433, 33.74727223])

**Use only somes informative features for exmploration between Langevin and VITS**

In [57]:
X, y, true_weights = regression_model(n_samples=200, n_features=20, n_informative=10, n_targets=1, 
 bias=0.0, effective_rank=None, 
 tail_strength=0.5, noise=0.0, 
 shuffle=True, coef=True, random_state=None)

In [58]:
true_weights

array([59.34531915,  0.        ,  0.        ,  0.        , 62.60412179,
        0.        , 68.41506501, 49.74881603, 49.19828199,  0.        ,
       27.61325967,  0.        , 25.02298735,  0.        ,  0.        ,
        0.        , 86.116001  , 84.25914542, 33.76544999,  0.        ])

**Use ill conditionned matrix**

In [59]:
X, y, true_weights = regression_model(n_samples=200, n_features=20, n_informative=20, n_targets=1, 
 bias=0.0, effective_rank=3, 
 tail_strength=0.5, noise=0.0, 
 shuffle=True, coef=True, random_state=None)

In [60]:
true_weights

array([82.63741387, 71.38216631,  2.74751575, 57.17781434,  1.15806037,
        2.78369081, 78.33314617, 85.94703388,  2.60762353, 82.27645594,
       60.86559265, 47.29645928, 68.26453904, 95.44293411,  3.57438985,
       24.11400692, 50.51530983, 15.06318527, 73.19534062, 51.13005124])

In [ ]:
class LMC():
    
    def __init__(self,dimention):
        
        self.dimention=dimention
        self.lbd=1
        self.eta=1
        
    def loglik(theta, y , x):
        V=self.lbd*np.eye(self.dimension) + x*x.T
        b=
        return (self.eta/2)*(theta.T*V*theta -2teheta.T*b + np.sum(y**2))
    
    def gradient(self,y,x):
        V=self.lbd*np.eye(self.dimension) + x*x.T
        b=
        return eta*(V*theta-b)
        
    def hessian(self,x):
        return self.eta*(self.lbd*np.eye(self.dimension) + x*x.T)
        
    def sample(self,T):
        
        eps = np.random.normal( shape=(self.dimension, 1))
        theta = (mean.T + T * eps).squeeze()
        return  theta     

    def update_mean(self, mean, gradient, h,T):
        return mean - h * gradient + T*noise
    
    def run(x,y,max_iter=10000):
        
        # initilialisation mean
        mean_0=np.zeros(dimention)
        dists=[]
           
        for t in range(max_iter):
            
            h=1/(t+1)
            # sample
            theta=sample(mean,y,x)
            # compute grad and hessian
            grad,hess=gradient(theta,y,x),hessian(x)
        
            # Update mean and cov
            mean=updade_mean(mean,grad,h)
            cov,cov_inv=update_cov(cov,cov_inv,hess,h)
            
            # compute the distance
            dists.append(np.norm(self.true_param-mean))
            
        return mean, cov ,dists
        

class VI_gaussian(true_param)

    def __init__(self,dimention,approx=False):
        
        self.dimention=dimention
        self.lbd=1
        self.eta=1
        self.approx=approx
        self.true_param=true_param
        
    
    def loglik(theta, y ,x):
        V=self.lbd*np.eye(self.dimension) + x*x.T
        b=
        return (self.eta/2)*(theta.T*V*theta -2teheta.T*b + np.sum(y**2))
    
    def gradient(self,y,x):
        V=self.lbd*np.eye(self.dimension) + x*x.T
        b=
        return eta*(V*theta-b)
        
    def hessian(self,x):
        return self.eta*(self.lbd*np.eye(self.dimension) + x*x.T)
    
    
    def update_mean(self, mean, gradient, h):
        return mean - h * gradient
    
    def update_cov(self, cov_semi, cov_semi_inv, hessian, h):
        cov_semi = (np.eye(self.dimension) - h * hessian) @ cov_semi + h * cov_semi_inv.T
        
        if self.approx:
            cov_semi_inv = cov_semi_inv @ (np.eye(self.dimension) - h * (np.matmul(cov_semi_inv.T , cov_semi_inv) - hessian))
        else:
            cov_semi_inv = np.linalg.inv(cov_semi)
        return cov_semi, cov_semi_inv
    
    def sample(self, mean, cov_semi):
        
        eps = np.random.normal( shape=(self.dimension, 1))
        theta = (mean.T + cov_semi @ eps).squeeze()
        return  theta
    
    def compute_cond_number(self, mean, cov_semi,label,features):
        features, labels, mean, cov_semi, _ = utils_vector
        theta = self.sample( mean, cov_semi)
        regularization_hessian = 2 * self.lbd * jnp.eye(self.dimension)
        data_hessian = np.sum(jax.vmap(self.hessian_function, in_axes=(None, 0, 0))(theta, features, labels), axis=0)
        hessian = self.eta *  (regularization_hessian + data_hessian)
        return np.linalg.cond(hessian)
    
    def run(x,y,max_iter=10000):
        
        # initilialisation, comment déjà pour la cov ?
        mean_0=np.zeros(dimention)
        cov=self.eta* np.eye(dimention)
        dists=[]
        cov_inv=(1/self.eta)* np.eye(dimention)
        
        for t in range(max_iter):
            h=1/(t+1)
            # sample
            theta=sample(mean,cov)
            # compute grad and hessian
            grad,hess=gradient(theta,y,x),hessian(x)
            
            # Update mean and cov
            mean=updade_mean(mean,grad,h)
            cov,cov_inv=update_cov(cov,cov_inv,hess,h)
            
            # compute the distance
            dists.append(np.norm(self.true_param-mean))
            
        return mean, cov ,dists
    
            

**Algorrithms** : Langevin : VITS